# Setup

Required libs:
* langchain: Building applications with LLMs through composability.
* ibm-watsonx-ai: ibm-watsonx-ai is a library that allows to work with watsonx.ai service on IBM Cloud and IBM Cloud for Data. Train, test, and deploy your models as APIs for application development and share with colleagues using this python library.
* langchain-ibm: This package provides the integration between LangChain and IBM watsonx.ai through the ibm-watsonx-ai SDK.
* unstructured: A library that prepares raw documents for downstream ML tasks.
* ibm-watson-machine-learning: A library that allows to work with Watson Machine Learning service on IBM Cloud and IBM Cloud for Data. Train, test, and deploy your models as APIs for application development and share with colleagues using this python library.

Install required libs

In [ ]:
%%capture
%pip install langchain==0.2.6 | tail -n 1
%pip install langchain_chroma==0.1.2 | tail -n 1
%pip install langchain-community==0.2.6 | tail -n 1
%pip install ibm-watsonx-ai==1.0.10 | tail -n 1
%pip install langchain_ibm==0.1.11 | tail -n 1
%pip install unstructured==0.15.0 | tail -n 1
%pip install ibm-watson-machine-learning==1.0.360 | tail -n 1

Import required libs

In [ ]:
#import required libraries
import os

from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes

from langchain_ibm import WatsonxEmbeddings, WatsonxLLM
from langchain.vectorstores import Chroma

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


import warnings
warnings.filterwarnings('ignore')

Credentials

In [ ]:
from ibm_watsonx_ai import Credentials
import os

credentials = Credentials(
                   url = "https://us-south.ml.cloud.ibm.com",
                  )
project_id = "skills-network"

## Indexing

the URLs

In [ ]:
import requests

class Document:
    def __init__(self, metadata, page_content):
        self.metadata = metadata
        self.page_content = page_content

URLS_DICTIONARY = {
    "watsonx_wiki": "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/PWMJ9-Npq9FYNSWrrf99YQ/watsonx.txt",
    "ibm_cloud_wiki": "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/wxekgOAVRH71dO92DEbwfQ/ibm-cloud.txt",
}
COLLECTION_NAME = "ibm_products"

documents = []

for name, url in URLS_DICTIONARY.items():
    print(f"Loading from {url}")
    response = requests.get(url)

    if response.status_code == 200:
        data = {
            'metadata': {"source": url, "name": name},
            'page_content': response.text
        }

        documents.append(Document(metadata=data['metadata'], page_content=data['page_content']))
        print(f"Loaded from {url}")
    else:
        print(f"Failed to retrieve content from {url}")


print(documents[0].metadata)
print(documents[0].page_content)

# get rid of white space
doc_id = 0
for doc in documents:
    doc.page_content = " ".join(doc.page_content.split()) # remove white space

    doc.metadata["id"] = doc_id #make a document id and add it to the document metadata

    print(doc.metadata)
    doc_id += 1

# Let's see how our sample document looks now after we cleaned it up.
display(documents[1].metadata)
display(documents[1].page_content)

## Store

Chunking: using `RecursiveCharacterTextSplitter` from LangChain

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Embedding with watsonx.ai

Alternatively, we can use the [Hugging Face embeddings models](https://python.langchain.com/v0.2/docs/integrations/platforms/huggingface/#embedding-models) via LangChain.

In [ ]:
embeddings = WatsonxEmbeddings(
    model_id=EmbeddingTypes.IBM_SLATE_30M_ENG.value,
    url=credentials["url"],
    project_id=project_id,
    )

vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

Test it with `similarity_search_with_score`: returns the documents and the (Euclide) distance score (lower score is better)

In [ ]:
query = "What is IBM?"
search = vectorstore.similarity_search_with_score(query, k=4)
search

## Retrieve & Generate response

* Setup a retriever
* Generate response
* Setup prompt template
* Define helper function: format_docs
* Setup a chain with context, prompt and llm model. Parse result with `StrOutputParser`

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={'k':2})

model_id = "meta-llama/llama-3-405b-instruct"

parameters = {
    GenParams.DECODING_METHOD: 'greedy',
    GenParams.MIN_NEW_TOKENS: 10,
    GenParams.MAX_NEW_TOKENS: 512,
    GenParams.REPETITION_PENALTY:1,
    GenParams.RETURN_OPTIONS: {'input_tokens': True,'generated_tokens': True, 'token_logprobs': True, 'token_ranks': True, }
}

# instantiate the LLM
llm = WatsonxLLM(
    model_id=model_id,
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id=project_id,
    params=parameters
)

template = """Generate a summary of the context that answers the question. Explain the answer in multiple steps if possible. 
Answer style should match the context. Ideal Answer Length 2-3 sentences.\n\n{context}\nQuestion: {question}\nAnswer:
"""
prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

Ask a question

In [ ]:
import pprint

pprint.pprint(chain.invoke("What is watsonx?"), width=120) 

Output:

("Watsonx is an artificial intelligence platform developed by IBM, named after Thomas J. Watson, the company's founder "
 'and first CEO. It includes multiple services such as content generation, summarization, text classification, and '
 'data extraction, and allows fine-tuning with its Tuning Studio. Additionally, Watsonx has a platform called '
 'Watsonx.data, which helps clients address data-related issues and facilitates seamless data access.')